In [1]:
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch3d
import pytorch3d.io
from pytorch3d.structures import Meshes
from pytorch3d.renderer import TexturesVertex

import trimesh

from src.util import make_faces
from src.cube_util import all_edge_triangles

In [29]:
n = 1001
start = -0.5
end = 0.5

d1, d2 = torch.meshgrid(
    torch.linspace(start, end, steps=n), 
    torch.linspace(start, end, steps=n))
d3 = torch.full_like(d1, end) + 1 / n
sides =  OrderedDict({
    'front': torch.stack((+d3,  d1,  d2), dim=-1),
    'right': torch.stack(( d1, +d3,  d2), dim=-1),    
    'back' : torch.stack((-d3,  d1,  d2), dim=-1),         
    'left' : torch.stack(( d1, -d3,  d2), dim=-1),
    'top'  : torch.stack(( d1,  d2, +d3), dim=-1),
    'down' : torch.stack(( d1,  d2, -d3), dim=-1),
})
vert = torch.stack(list(sides.values())).reshape(-1, 3)
offset, faces = n ** 2, make_faces(n, n)
faces = torch.cat([
    i * offset + torch.tensor(faces)
    for i in range(6)])
textures = TexturesVertex(verts_features=[torch.ones_like(vert)])
mesh = Meshes(verts=[vert], faces=[faces], textures=textures)

edge_faces = torch.tensor(all_edge_triangles(n))

def get_corners(n):
    s0 = 0
    s1 = n-1
    s2 = n**2-n
    s3 = n**2-1
    tris = torch.tensor([
        [0, 5, 3],
        [0, 4, 1],
        [4, 2, 3],
        [4, 2, 1],
        [0, 4, 3],
        [0, 5, 1],
        [5, 2, 3],
        [5, 2, 1]])    
    rmn = torch.tensor([
        [s0, s2, s2],
        [s3, s3, s3],
        [s0, s1, s1],
        [s1, s3, s1],
        [s1, s2, s3],
        [s2, s3, s2],
        [s0, s0, s0],
        [s1, s2, s0]])
    return tris * n**2+ rmn

corners = get_corners(n)
m = trimesh.Trimesh(vertices=vert, 
    faces=torch.cat((faces, 
                     corners,
                     edge_faces,
                    )))
filename = f'./data/cube-{n}x{n}.stl'
m.export(filename);
filename

'./data/cube-1001x1001.stl'

In [22]:
corners

tensor([[ 0, 30, 30],
        [35, 35, 35],
        [ 0,  5,  5],
        [ 5, 35,  5],
        [ 5, 30, 35],
        [30, 35, 30],
        [ 0,  0,  0],
        [ 5, 30,  0]])

In [3]:
corners = torch.tensor([
    [ 0, 22, 14],
    [ 3, 19,  7],
    
    [16,  9, 13],   
    [17, 11,  5],
    
    [ 1, 18, 15],
    [ 2, 23,  6],
    
    [20,  8, 12],
    [21, 10,  4],
])
corners

tensor([[ 0, 22, 14],
        [ 3, 19,  7],
        [16,  9, 13],
        [17, 11,  5],
        [ 1, 18, 15],
        [ 2, 23,  6],
        [20,  8, 12],
        [21, 10,  4]])

In [4]:
tris = torch.div(corners, 4, rounding_mode='trunc')
tris

tensor([[0, 5, 3],
        [0, 4, 1],
        [4, 2, 3],
        [4, 2, 1],
        [0, 4, 3],
        [0, 5, 1],
        [5, 2, 3],
        [5, 2, 1]])

In [5]:
torch.remainder(corners, 4)

tensor([[0, 2, 2],
        [3, 3, 3],
        [0, 1, 1],
        [1, 3, 1],
        [1, 2, 3],
        [2, 3, 2],
        [0, 0, 0],
        [1, 2, 0]])

In [16]:
n = 5
s0 = 0
s1 = n-1
s2 = n**2-n
s3 = n**2-1
print(torch.tensor([
    [s1, s3],
    [s0, s2],
]))
rmn = torch.tensor([
    [s0, s2, s2],
    [s3, s3, s3],
    [s0, s1, s1],
    [s1, s3, s1],
    [s1, s2, s3],
    [s2, s3, s2],
    [s0, s0, s0],
    [s1, s2, s0]])
print(rmn)

tensor([[ 4, 24],
        [ 0, 20]])
tensor([[ 0, 20, 20],
        [24, 24, 24],
        [ 0,  4,  4],
        [ 4, 24,  4],
        [ 4, 20, 24],
        [20, 24, 20],
        [ 0,  0,  0],
        [ 4, 20,  0]])


In [23]:
tris

tensor([[0, 5, 3],
        [0, 4, 1],
        [4, 2, 3],
        [4, 2, 1],
        [0, 4, 3],
        [0, 5, 1],
        [5, 2, 3],
        [5, 2, 1]])

In [25]:
def get_corners(n):
    s0 = 0
    s1 = n-1
    s2 = n**2-n
    s3 = n**2-1
    tris = torch.tensor([
        [0, 5, 3],
        [0, 4, 1],
        [4, 2, 3],
        [4, 2, 1],
        [0, 4, 3],
        [0, 5, 1],
        [5, 2, 3],
        [5, 2, 1]])    
    rmn = torch.tensor([
        [s0, s2, s2],
        [s3, s3, s3],
        [s0, s1, s1],
        [s1, s3, s1],
        [s1, s2, s3],
        [s2, s3, s2],
        [s0, s0, s0],
        [s1, s2, s0]])
    return tris * n**2+ rmn

get_corners(3)

tensor([[ 0, 51, 33],
        [ 8, 44, 17],
        [36, 20, 29],
        [38, 26, 11],
        [ 2, 42, 35],
        [ 6, 53, 15],
        [45, 18, 27],
        [47, 24,  9]])

In [ ]:
tris